# 국민청원 분류하기

- 페이지 크롤링
- 전처리 
- 단어임베딩
- textCNN

# 크롤링 데이터 읽어오기

In [115]:
import pandas as pd
import numpy as np
import glob
import os

csv_files = glob.glob(os.path.join('./crawl', "*.csv"))    
df_from_each_file = (pd.read_csv(f) for f in csv_files)
df = pd.concat(df_from_each_file, ignore_index=True)
print(len(df))
df.head()

8438


,start,end,category,count,title,content
0,2020-06-08,2020-07-08,인권/성평등,185,공문서와 인장을 불법위조를 하여 범죄혐의자를 무혐의처리 되도록 가짜문건을 만들어 행...,저는 전 정부하에 2013년 8월 12일\r\n수면제를 이용하여 범죄를 저지른 범죄...
1,2020-06-08,2020-07-08,기타,883,방과후 강사들을 도와주세요.,초등학교 방과후 강사로 일하고 있는 사람입니다.\r\n지금 모든 학교가 온라인과 등...
2,2020-06-08,2020-07-08,성장동력,170,과적.심청이 아부지법,존경할수 없는 대한민국 국회의원 법관 그리고 이나라의 대표 이신 대통령 저는 젊어서...
3,2020-06-08,2020-07-08,육아/교육,779,대통령님 9살 아들죽인 계모때문에라도 사형제도 부활시켜주세요,대통령님\r\n지금 현재 대한민국이라는 나라는 처벌의 약함을 빌미로 \r\n사람을 ...
4,2020-06-08,2020-07-08,반려동물,"19,027",계양산개농장철거!!!,안녕하세요!! 계양산으로 등산을 자주 가기도 했는데 뉴스로 충격적인 기사를 접해서 ...


# 데이터 전처리 

In [116]:
df.loc[1]['content'][:500]

'초등학교 방과후 강사로 일하고 있는 사람입니다.\r\n지금 모든 학교가 온라인과 등교 수업을 겸하고 있지만... 방과후 강사들의 일자리인 학교에서 당분간은 수업 진행이 힘들다는 통보만 받았습니다.1학기가 지나고 2학기가 시작된다 하여도 지금 현재로서는 어떠한 계획도 없는 상황이라고 합니다. 학교에서도 방과후 선생님들을 위해 등교도우미와 같은 일자리를 제공하고 있다고는 하지만 그것 조차도 뽑는 인원이 턱없이 적고..보수도 주14시간 미만으로만 일 할 수 있기에 방과후선생님들이 처한 경제적인 어려움에는 크게 도움이 되지 않습니다.\r\n저와 같이 생업으로 이 일을 하고 있는 많은 선생님들께서도 당장 매달 지출해야 하는 대출금이나 교육비 카드값등 나가는 돈들은 정해져 있는데...수입이 전혀 없는 상태로 지난 3개월을 버티는것 조차 쉬운일이 아니였습니다. 더는 버틸 수도 없고 앞날도 막막해진 이 시점에서 국민여러분들의 도움을 받고자 이 글을 쓰게 되었습니다.\r\n방과후 강사는 프리랜서 이기에 은행권'

In [117]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))   # \f :form feed   \v : vertical tab
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))  # 자음 : ㄱ-ㅎ, 모음 : ㅏ-ㅣ , 한글자:  가-힣, 숫자:0-9  이외에 제거 
    return text

remove_special_char(remove_white_space(df.loc[1]['content']))[:500]

'초등학교 방과후 강사로 일하고 있는 사람입니다   지금 모든 학교가 온라인과 등교 수업을 겸하고 있지만  방과후 강사들의 일자리인 학교에서 당분간은 수업 진행이 힘들다는 통보만 받았습니다 1학기가 지나고 2학기가 시작된다 하여도 지금 현재로서는 어떠한 계획도 없는 상황이라고 합니다  학교에서도 방과후 선생님들을 위해 등교도우미와 같은 일자리를 제공하고 있다고는 하지만 그것 조차도 뽑는 인원이 턱없이 적고 보수도 주14시간 미만으로만 일 할 수 있기에 방과후선생님들이 처한 경제적인 어려움에는 크게 도움이 되지 않습니다   저와 같이 생업으로 이 일을 하고 있는 많은 선생님들께서도 당장 매달 지출해야 하는 대출금이나 교육비 카드값등 나가는 돈들은 정해져 있는데 수입이 전혀 없는 상태로 지난 3개월을 버티는것 조차 쉬운일이 아니였습니다  더는 버틸 수도 없고 앞날도 막막해진 이 시점에서 국민여러분들의 도움을 받고자 이 글을 쓰게 되었습니다   방과후 강사는 프리랜서 이기에 은행권 대출은 '

In [118]:
df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

# 토크나이징 및 변수 생성

제목은 형태소 단위, 내용은 명사 단위, 명사만 추출해서 학습

참고 : KoNLPy 한국어 처리 패키지
* https://datascienceschool.net/03%20machine%20learning/03.01.02%20KoNLPy%20%ED%95%9C%EA%B5%AD%EC%96%B4%20%EC%B2%98%EB%A6%AC%20%ED%8C%A8%ED%82%A4%EC%A7%80.html

[토크나이징]

In [119]:
from konlpy.tag import Okt
okt = Okt()
df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

In [123]:
df.head(1)

,start,end,category,count,title,content,title_token,content_token
0,2020-06-08,2020-07-08,인권/성평등,185,공문서와 인장을 불법위조를 하여 범죄혐의자를 무혐의처리 되도록 가짜문건을 만들어 행...,저는 전 정부하에 2013년 8월 12일 수면제를 이용하여 범죄를 저지른 범죄혐의...,"[공문서, 와, 인장, 을, 불법, 위조, 를, 하여, 범죄, 혐의, 자, 를, 무...","[저, 전, 정부, 수면제, 이용, 범죄, 범죄, 혐의, 명, 지방검찰청, 고소, ..."


In [122]:
#df.to_pickle('temp/tokenized.pkl')   
df = pd.read_pickle('temp/tokenized.pkl')

[파생변수 생성]

[pandas serise string replace with regex](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html)

In [124]:
df['token_final'] = df.title_token + df.content_token
df['count'] = df['count'].replace({',':''}, regex = True).apply(lambda x: int(x))   # 문자화된 숫자를 숫자로 변경
df['label'] = df['count'].apply(lambda x: 'Yes' if x >=1000 else 'No')

df_drop = df[['token_final', 'label']]  # 머신러닝에 쓸 것만 

In [125]:
df_drop.head()

,token_final,label
0,"[공문서, 와, 인장, 을, 불법, 위조, 를, 하여, 범죄, 혐의, 자, 를, 무...",No
1,"[방과후, 강사, 들, 을, 도와주세요, 초등학교, 방과후, 강사, 일, 사람, 지...",No
2,"[과, 적, 심청, 이, 아부, 지법, 존경, 대한민국, 국회의원, 법관, 이나라,...",No
3,"[대통령, 님, 9, 살, 아들, 죽인, 계모, 때문, 에라도, 사형제, 도, 부활...",No
4,"[계양산, 개, 농장, 철거, 계양산, 등산, 자주, 가기, 뉴스, 충격, 기사, ...",Yes


[단어 임베딩] : [gensim.Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) skipgram

In [127]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# 생성
embedding_model = Word2Vec(
    df_drop['token_final'], 
    sg = 1, # skip-gram
    size = 100,
    window = 2,
    min_count = 1,
    workers = 4
)
print(embedding_model)

# 저장
embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v')

# 사용
loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v')
model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec(vocab=40581, size=100, alpha=0.025)
[('음주', 0.8935098052024841), ('뺑소니', 0.848153829574585), ('살인마', 0.7900915741920471), ('무면허', 0.789531409740448), ('절도', 0.7880804538726807), ('스토킹', 0.7821337580680847), ('동물학대', 0.7699282169342041), ('강력범죄', 0.7664657831192017), ('검거', 0.7615181803703308), ('싸이코패스', 0.7603848576545715)]


[데이터셋 분할 및 저장]

In [128]:
from numpy.random import RandomState
rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

assert tr.index.size + val.index.size == len(df_drop)

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

[Field 클래스 정의] : torchtext.Field
 - [use legacy](https://github.com/pytorch/text/blob/master/examples/legacy_tutorial/migration_tutorial.ipynb)

[dataset 만들기] [torchtext.data](https://torchtext.readthedocs.io/en/latest/data.html)

In [129]:
import torchtext
from torchtext.legacy.data import Field, TabularDataset

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))   # 대괄호, ' 제거
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print('Train: {}, {}'.format(train[0].text[:10], train[0].label))
print('validation: {} {}'.format(validation[0].text[:10], validation[0].label))

Train: ['들', '방통위', '을', '처벌', '해주세요', '개그우먼', '씨', '나', '혼자', '방송'], No
validation: ['사업', '부도', '와', '국세', '행정', '안녕', '격무', '노고', '저', '최'] No


[단어장 및 datasetloader 정의] : [torchtext.vocab](https://torchtext.readthedocs.io/en/latest/vocab.html)

In [130]:
import torch
from torchtext.legacy.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors = Vectors(name='data/petitions_tokens_w2v')
TEXT.build_vocab(train, # dataset
                 # followings are passed to constructor of Vocab
                 vectors = vectors, min_freq = 1, max_size = None)

LABEL.build_vocab(train)
vocab = TEXT.vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('embedding dim: {}'.format(TEXT.vocab.vectors.shape))

embedding dim: torch.Size([35644, 100])


[TextCNN 모델링]

In [131]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)   # 기 형성된 word2vec 데이터 셋팅
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])   # conv2d variable-kernel parallel
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)
        
    def forward(self, x):
        # x: (batch, max_len)

        emb_x = self.embed(x)   # (batch, max_len, emb_dim)
        emb_x = emb_x.unsqueeze(1)  # (batch, 1, max_len, emb_dim)

        conv_x = [self.relu(conv(emb_x)) for conv in self.convs]   # 커널수만큼  parallel, (batch, out_channel=10, max_len', 1)
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in conv_x]    # 각 channel 에서 max element 1개 추출 
                                                                               # (batch, out_channel, 1)
        
        fc_x = torch.cat(pool_x, dim=1) # (batch, n_kernel*out_channel, 1)
        fc_x = fc_x.squeeze(-1) # (batch, n_kernel*out_channel)
        fc_x = self.dropout(fc_x)
        
        logit = self.fc(fc_x)  # (batch, n_kernel*out_channel)

        return logit

[모델 학습/평가 함수]

In [132]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

[모델 학습 및 성능 확인]

In [134]:
model = TextCNN(vocab, 100, 10, [3,4,5], 2).to(device)
#print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(model.parameters(), lr=0.001)
best_test_acc = -1
for epoch in range(1, 10):
    
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
    
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_BEST_validation.model")
        
    print('--------------------------------------------------------')
        

Train Epoch: 1 	 Loss: 0.08552285376301519 	 Accuracy: 56.20740509033203%
Valid Epoch: 1 	 Loss: 0.08501673157887436 	 Accuracy: 57.34597396850586%
model saves at 57.34597396850586 accuracy
--------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08295263561054513 	 Accuracy: 59.762962341308594%
Valid Epoch: 2 	 Loss: 0.079980618986897 	 Accuracy: 62.500003814697266%
model saves at 62.500003814697266 accuracy
--------------------------------------------------------
Train Epoch: 3 	 Loss: 0.064539615171927 	 Accuracy: 75.37777709960938%
Valid Epoch: 3 	 Loss: 0.08391831311180976 	 Accuracy: 62.914695739746094%
model saves at 62.914695739746094 accuracy
--------------------------------------------------------
Train Epoch: 4 	 Loss: 0.031169164081552514 	 Accuracy: 90.45925903320312%
Valid Epoch: 4 	 Loss: 0.10983274719055512 	 Accuracy: 62.44076156616211%
--------------------------------------------------------
Train Epoch: 5 	 Loss: 0.013412517882780068 	 Accura